In [ ]:
# install dependencies
!pip install -U torch torchvision cython
!pip install -U 'git+https://github.com/facebookresearch/fvcore.git' 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
torch.__version__

  Cloning https://github.com/facebookresearch/fvcore.git to /tmp/pip-req-build-uhsgakdi
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/fvcore.git /tmp/pip-req-build-uhsgakdi
  Resolved https://github.com/facebookresearch/fvcore.git to commit a491d5b9a06746f387aca2f1f9c7c7f28e20bef9
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-wa2b0c06
  Running command git clone --filter=blob:none --quiet https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-wa2b0c06
  Resolved https://github.com/cocodataset/cocoapi.git to commit 8c9bcc3cf640524c4c20a9c40e89cb6a2f2fa0e9
  Preparing metadata (setup.py) ... done
  Created wheel for fvcore: filename=fvcore-0.1.6-py3-none-any.whl size=65670 sha256=59c926bba38e77dbd24acf2b3f47bc891093c0797f9eaf8105af076729c04055
  Stored in directory: /tmp/pip-ephem-wheel-cache-yffbnosw/wheels/8f/cb/6a/3b7ac0e01781855ca3d1417ebf9e15e20d5b7fe37ab063aa50


'2.5.1+cu121'

In [ ]:
!git clone https://github.com/facebookresearch/detectron2 detectron2_repo
!pip install -e detectron2_repo

fatal: destination path 'detectron2_repo' already exists and is not an empty directory.
Obtaining file:///content/detectron2_repo
  Preparing metadata (setup.py) ... done
  Using cached pycocotools-2.0.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.1 kB)
  Using cached fvcore-0.1.5.post20221221-py3-none-any.whl
Using cached pycocotools-2.0.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (427 kB)
  Attempting uninstall: fvcore
    Found existing installation: fvcore 0.1.6
    Uninstalling fvcore-0.1.6:
      Successfully uninstalled fvcore-0.1.6
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0
  Attempting uninstall: detectron2
    Found existing installation: detectron2 0.6
    Uninstalling detectron2-0.6:
      Successfully uninstalled detectron2-0.6
  Running setup.py develop for detectron2


In [ ]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

In [ ]:
from google.colab import drive

# Google Drive 마운트
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os
import json

def merge_json_files(json_dir, image_dir, output_json):
    """
    여러 개의 JSON 파일을 COCO 형식으로 병합.
    Args:
        json_dir: JSON 파일들이 저장된 디렉토리 경로
        image_dir: 이미지 파일들이 저장된 디렉토리 경로
        output_json: 병합된 JSON 파일을 저장할 경로
    """
    coco_format = {
        "images": [],
        "annotations": [],
        "categories": []
    }
    annotation_id = 1
    category_mapping = {}

    # JSON 파일 처리
    for json_file in os.listdir(json_dir):
        if not json_file.endswith('.json'):
            continue

        with open(os.path.join(json_dir, json_file), 'r') as f:
            data = json.load(f)

        # 이미지 정보 추가
        image_info = data["images"][0]  # 단일 이미지
        coco_format["images"].append(image_info)

        # 주석 정보 추가
        for annotation in data["annotations"]:
            annotation["id"] = annotation_id
            annotation_id += 1
            coco_format["annotations"].append(annotation)

        # 카테고리 추가
        for category in data["categories"]:
            if category["id"] not in category_mapping:
                coco_format["categories"].append(category)
                category_mapping[category["id"]] = category["name"]

    # COCO 형식 JSON 저장
    with open(output_json, 'w') as f:
        json.dump(coco_format, f, indent=4)

# 데이터셋 병합 실행
merge_json_files(
    json_dir="/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/train/json",
    image_dir="/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/train/images",
    output_json="/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/train/train_coco.json"
)
merge_json_files(
    json_dir="/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/val/json",
    image_dir="/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/val/images",
    output_json="/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/val/val_coco.json"
)
merge_json_files(
    json_dir="/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/test/json",
    image_dir="/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/test/images",
    output_json="/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/test/test_coco.json"
)


In [ ]:
from detectron2.data.datasets import register_coco_instances

# 데이터셋 등록
register_coco_instances("train_dataset", {}, "/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/train/train_coco.json", "/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/train/images")
register_coco_instances("val_dataset", {}, "/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/val/val_coco.json", "/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/val/images")
register_coco_instances("test_dataset", {}, "/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/test/test_coco.json", "/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/test/images")


In [ ]:
import os

# 이미지 폴더 경로
image_dir = '/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/val/images'

# 이미지 파일 리스트
image_files = os.listdir(image_dir)

# 파일 이름 수정 및 경로 수정
for image_file in image_files:
    if image_file.endswith("jpg.jpg"):  # 현재 파일명 패턴 확인
        # 불필요한 부분을 제거하여 새로운 파일명 생성
        corrected_name = image_file.replace('jpg.jpg', '.jpg')

        old_path = os.path.join(image_dir, image_file)
        new_path = os.path.join(image_dir, corrected_name)

        # 파일 이름 수정
        os.rename(old_path, new_path)
        print(f"Renamed: {image_file} to {corrected_name}")



Renamed: 04_03_20_6000236jpg.jpg to 04_03_20_6000236.jpg
Renamed: 04_03_20_6000415jpg.jpg to 04_03_20_6000415.jpg
Renamed: 04_03_20_6000124jpg.jpg to 04_03_20_6000124.jpg
Renamed: 04_03_20_6000667jpg.jpg to 04_03_20_6000667.jpg
Renamed: 04_03_20_6001176jpg.jpg to 04_03_20_6001176.jpg
Renamed: 04_03_20_6000977jpg.jpg to 04_03_20_6000977.jpg
Renamed: 04_03_20_6000160jpg.jpg to 04_03_20_6000160.jpg
Renamed: 04_03_20_6000455jpg.jpg to 04_03_20_6000455.jpg
Renamed: 04_03_20_6000123jpg.jpg to 04_03_20_6000123.jpg
Renamed: 04_03_20_6000018jpg.jpg to 04_03_20_6000018.jpg
Renamed: 04_03_20_6000666jpg.jpg to 04_03_20_6000666.jpg
Renamed: 04_03_20_6000644jpg.jpg to 04_03_20_6000644.jpg
Renamed: 04_03_20_6000992jpg.jpg to 04_03_20_6000992.jpg
Renamed: 04_03_20_6001160jpg.jpg to 04_03_20_6001160.jpg
Renamed: 04_03_20_6001037jpg.jpg to 04_03_20_6001037.jpg
Renamed: 04_03_20_6000570jpg.jpg to 04_03_20_6000570.jpg
Renamed: 04_03_20_6000757jpg.jpg to 04_03_20_6000757.jpg
Renamed: 04_03_20_6001190jpg.jp

In [ ]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultTrainer
import json

def get_num_classes(coco_json_path):
    """
    COCO JSON 파일에서 카테고리 개수를 반환.
    Args:
        coco_json_path (str): COCO JSON 파일 경로
    Returns:
        int: 카테고리 개수
    """
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)
    return len(coco_data["categories"])

# train_coco.json에서 카테고리 개수 읽기
num_classes = get_num_classes("/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/train/train_coco.json")

# Config 설정
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/Dataset/자연재해로 인한 생활시설 안전 데이터/112.자연재해로 인한 생활시설 안전 데이터(AI모델)/① 모델소스코드/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.DATASETS.TRAIN=("train_dataset", )
cfg.DATASETS.TEST=("test_dataset", )
cfg.DATALOADER.NUM_WORKERS = 4
cfg.MODEL.WEIGHTS="detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes

# 모델 훈련
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

[11/28 03:41:39 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

roi_heads.box_predictor.bbox_pred.{bias, weight}
roi_heads.box_predictor.cls_score.{bias, weight}
roi_heads.mask_head.predictor.{bias, weight}


[11/28 03:41:39 d2.engine.train_loop]: Starting training from iteration 0


/usr/local/lib/python3.10/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[11/28 03:41:56 d2.utils.events]:  eta: 0:09:22  iter: 19  total_loss: 2.648  loss_cls: 1.326  loss_box_reg: 0.5598  loss_mask: 0.6995  loss_rpn_cls: 0.02128  loss_rpn_loc: 0.03294    time: 0.5800  last_time: 0.5290  data_time: 0.0708  last_data_time: 0.0111   lr: 4.9953e-06  max_mem: 2753M
[11/28 03:42:11 d2.utils.events]:  eta: 0:08:56  iter: 39  total_loss: 2.506  loss_cls: 1.253  loss_box_reg: 0.4735  loss_mask: 0.6926  loss_rpn_cls: 0.01399  loss_rpn_loc: 0.03313    time: 0.5730  last_time: 0.4538  data_time: 0.0155  last_data_time: 0.0063   lr: 9.9902e-06  max_mem: 2753M
[11/28 03:42:22 d2.utils.events]:  eta: 0:08:44  iter: 59  total_loss: 2.351  loss_cls: 1.127  loss_box_reg: 0.4577  loss_mask: 0.6848  loss_rpn_cls: 0.01752  loss_rpn_loc: 0.01953    time: 0.5675  last_time: 0.6198  data_time: 0.0082  last_data_time: 0.0171   lr: 1.4985e-05  max_mem: 2753M
[11/28 03:42:33 d2.utils.events]:  eta: 0:08:30  iter: 79  total_loss: 2.113  loss_cls: 0.9229  loss_box_reg: 0.5356  loss_m

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
import cv2
import json

# 저장된 모델 경로
model_weights_path = "/content/output/model_final.pth"

# COCO JSON 파일에서 카테고리 개수 읽기
def get_num_classes(coco_json_path):
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)
    return len(coco_data["categories"])

num_classes = get_num_classes("/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/train/train_coco.json")

# Config 설정
cfg = get_cfg()
cfg.merge_from_file("/content/drive/MyDrive/Dataset/자연재해로 인한 생활시설 안전 데이터/112.자연재해로 인한 생활시설 안전 데이터(AI모델)/① 모델소스코드/detectron2/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.WEIGHTS = model_weights_path
cfg.MODEL.ROI_HEADS.NUM_CLASSES = num_classes  # num_classes 설정
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # 추론 시 threshold 설정
cfg.DATASETS.TEST = ("test_dataset",)

# Predictor 생성
predictor = DefaultPredictor(cfg)

# 예시 이미지 경로
image_path = '/content/drive/MyDrive/Dataset/dataset/GDSC_Sprint_Challenge/dataset/test/images/04_03_20_6000049.jpg'

# 이미지 읽기
im = cv2.imread(image_path)

# 추론
outputs = predictor(im)

# 심각도 측정 기준 (확신도에 따라)
def measure_severity(scores):
    severity = []
    for score in scores:
        if score > 0.9:
            severity.append("High Severity")
        elif score > 0.5:
            severity.append("Medium Severity")
        else:
            severity.append("Low Severity")
    return severity

# 예측 결과의 확신도 (scores) 가져오기
scores = outputs["instances"].scores

# 심각도 측정
severity = measure_severity(scores)

# 결과 출력
for idx, score in enumerate(scores):
    print(f"Object {idx + 1}: {severity[idx]} (Confidence Score: {score:.2f})")

# 결과 시각화
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get("test_dataset"), scale=1.2)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
result_image = v.get_image()[:, :, ::-1]

# 결과 이미지 저장
cv2.imwrite("/content/output/result_image_04_03_20_6000049.jpg", result_image)


[11/28 03:59:05 d2.checkpoint.detection_checkpoint]: [DetectionCheckpointer] Loading from /content/output/model_final.pth ...


/usr/local/lib/python3.10/dist-packages/fvcore/common/checkpoint.py:252: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=torch.device("cpu"))

Object 1: Medium Severity (Confidence Score: 0.82)


True